In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  2 21:55:50 2019

@author: souviksamanta
"""
#Importing Required Libraries
import pandas as pd
import numpy as np
import string
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')


# my_analyzer replaces both the preprocessor and tokenizer and replaces stop word removal and ngram constructions

def my_analyzer(s):
    # Synonym List
    syns = {'veh': 'vehicle', 'car': 'vehicle', 'chev':'cheverolet', \
              'chevy':'cheverolet', 'air bag': 'airbag', \
              'seat belt':'seatbelt', "n't":'not', 'to30':'to 30', \
              'wont':'would not', 'cant':'can not', 'cannot':'can not', \
              'couldnt':'could not', 'shouldnt':'should not', \
              'wouldnt':'would not', }
    
    # Preprocess String s
    s = s.lower()
    s = s.replace(',', '. ')
    # Tokenize 
    tokens = word_tokenize(s)
    tokens = [word.replace(',','') for word in tokens ]
    tokens = [word for word in tokens if ('*' not in word) and \
              ("''" != word) and ("``" != word) and \
              (word!='description') and (word !='dtype') \
              and (word != 'object') and (word!="'s")]
    
    # Map synonyms
    for i in range(len(tokens)):
        if tokens[i] in syns:
            tokens[i] = syns[tokens[i]]
            
    # Remove stop words
    punctuation = list(string.punctuation)+['..', '...']
    pronouns = ['i', 'he', 'she', 'it', 'him', 'they', 'we', 'us', 'them']
    stop = stopwords.words('english') + punctuation + pronouns
    filtered_terms = [word for word in tokens if (word not in stop) and \
                  (len(word)>1) and (not word.replace('.','',1).isnumeric()) \
                  and (not word.replace("'",'',2).isnumeric())]
    
    # Lemmatization & Stemming - Stemming with WordNet POS
    # Since lemmatization requires POS need to set POS
    tagged_words = pos_tag(filtered_terms, lang='eng')
    # Stemming with for terms without WordNet POS
    stemmer = SnowballStemmer("english")
    wn_tags = {'N':wn.NOUN, 'J':wn.ADJ, 'V':wn.VERB, 'R':wn.ADV}
    wnl = WordNetLemmatizer()
    stemmed_tokens = []
    for tagged_token in tagged_words:
        term = tagged_token[0]
        pos  = tagged_token[1]
        pos  = pos[0]
        try:
            pos   = wn_tags[pos]
            stemmed_tokens.append(wnl.lemmatize(term, pos=pos))
        except:
            stemmed_tokens.append(stemmer.stem(term))
    return stemmed_tokens

# Further Customization of Stopping and Stemming using NLTK
def my_preprocessor(s):
    #Vectorizer sends one string at a time
    s = s.lower()
    s = s.replace(',', '. ')
    print("preprocessor")
    return(s)
    
def my_tokenizer(s):
    # Tokenize
    print("Tokenizer")
    tokens = word_tokenize(s)
    tokens = [word.replace(',','') for word in tokens ]
    tokens = [word for word in tokens if word.find('*')!=True and \
              word != "''" and word !="``" and word!='description' \
              and word !='dtype']
    return tokens

# Increase Pandas column width to let pandas read large text columns
pd.set_option('max_colwidth', 32575)
# Read GMC Ignition Recall Comments from NTHSA Data

df = pd.read_excel("C:/Users/saman/OneDrive/Texas A&M/656/Python/CaliforniaCabernet.xlsx")

# Setup simple constants
n_docs     = len(df['description'])
n_samples  = n_docs
m_features = None
s_words    = 'english'
ngram = (1,2)

# Setup reviews in list 'discussions'
discussions = []
for i in range(n_samples):
    discussions.append(("%s" %df['description'].iloc[i]))
    
    


# Create Word Frequency by Review Matrix using Custom Analyzer
cv = CountVectorizer(max_df=0.95, min_df=2, max_features=m_features,\
                     analyzer=my_analyzer, ngram_range=ngram)
tf = cv.fit_transform(discussions)

print("\nVectorizer Parameters\n", cv, "\n")


n_topics        = 9
max_iter        =  5
learning_offset = 20.
learning_method = 'online'

# LDA for TF-IDF x Doc Matrix
# First Create Term-Frequency/Inverse Doc Frequency by Review Matrix
# This requires constructing Term Freq. x Doc. matrix first

tf_idf = TfidfTransformer()
print("\nTF-IDF Parameters\n", tf_idf.get_params(),"\n")
tf_idf = tf_idf.fit_transform(tf)

# Or you can construct the TF/IDF matrix from the data
tfidf_vect = TfidfVectorizer(max_df=0.95, min_df=2, max_features=m_features,\
                             analyzer=my_analyzer, ngram_range=ngram)
tf_idf = tfidf_vect.fit_transform(discussions)
print("\nTF_IDF Vectorizer Parameters\n", tfidf_vect, "\n")

lda = LatentDirichletAllocation(n_components=n_topics, max_iter=max_iter,\
                                learning_method=learning_method, \
                                learning_offset=learning_offset, \
                                random_state=12345)
lda.fit_transform(tf_idf)
print('{:.<22s}{:>6d}'.format("Number of Reviews", tf.shape[0]))
print('{:.<22s}{:>6d}'.format("Number of Terms",     tf.shape[1]))
print("\nTopics Identified using LDA with TF_IDF")
tf_features = cv.get_feature_names()
max_words = 15
topic_description=[]
for topic_idx, topic in enumerate(lda.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([tf_features[i]
                             for i in topic.argsort()[:-max_words - 1:-1]])
        topic_description.append(message[10:])
        print(message)
        print()
        
for i in range(len(topic_description)):
    topic_description[i]=topic_description[i].split(' ')
    
temp=lda.transform(tf_idf)
temp1=[]
for i in range(len(temp)):
    temp1.append(temp[i].argmax())
temp1=pd.DataFrame(temp1,columns=['Topic#'])
df=df.join(temp1)

df.head()

table1=df.pivot_table(['points','price'],index='Topic#')
table1=table1.join(pd.DataFrame(topic_description))

#Table 1 with average points, average price and 15 words of cluster description
table1=table1.rename_axis({'points':'avg_points','price':'avg_price'},axis=1)
table1.T

table2=df.pivot_table('Review',index='Region',columns='Topic#',\
                      aggfunc='count',\
                      fill_value=0,margins=True)
#Table 2 with Percent of reviews in that region associated with each of 9 clusters
def percent_convert(x):
    for index in x.index:
        for i in x.columns:
            x.loc[index,i]=round(x.loc[index,i]*100/x.loc[index,'All'],2)
    
    return x
percent_convert(table2)
table2.T


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\saman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Vectorizer Parameters
 CountVectorizer(analyzer=<function my_analyzer at 0x000002A0AFC87378>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.95,
        max_features=None, min_df=2, ngram_range=(1, 2), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None) 


TF-IDF Parameters
 {'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': False, 'use_idf': True} 


TF_IDF Vectorizer Parameters
 TfidfVectorizer(analyzer=<function my_analyzer at 0x000002A0AFC87378>,
        binary=False, decode_error='strict', dtype=<class 'numpy.float64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.95,
        max_features=None, min_df=2, ngram_range=(1, 2), norm='l2',
        preprocessor=None, smooth_idf=True, stop_words=None,
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:185: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead


Region,California Other,Central Coast,Central Valley,Clear Lake,High Valley,Lake County,Mendocino,Mendocino County,Mendocino Ridge,Mendocino/Lake Counties,Napa,Napa-Sonoma,North Coast,Red Hills Lake County,Redwood Valley,Sierra Foothills,Sonoma,South Coast,All
Topic#,,,,,,,,,,,,,,,,,,,
0,26.77,50.70,33.99,0.0,0.0,50.0,60.00,62.07,66.67,56.12,78.31,70.24,36.07,64.86,66.67,48.41,65.22,42.31,67.07
1,0.00,0.17,0.99,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,1.09,0.00,0.00,0.00,0.31,0.00,0.11
2,0.00,0.28,0.99,0.0,0.0,0.0,3.33,0.00,0.00,0.51,0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.14
3,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02
4,71.22,43.62,61.58,100.0,100.0,50.0,36.67,34.48,33.33,42.86,18.43,21.43,58.47,35.14,33.33,45.24,30.79,44.23,29.19
5,1.34,5.00,2.46,0.0,0.0,0.0,0.00,3.45,0.00,0.51,2.98,8.33,4.37,0.00,0.00,5.56,3.29,13.46,3.27
6,0.27,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.79,0.22,0.00,0.08
7,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.18,0.00,0.06
8,0.40,0.22,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.07
